# Data Cleaning and Preparation

In [1]:
import numpy as np
import pandas as pd


## Data Transformation (continue)
So far in this lesson we’ve been concerned with rearranging data. Filtering, cleaning,
and other transformations are another class of important operations.

### Detecting and Filtering Outliers
Filtering or transforming outliers is largely a matter of applying array operations.
Consider a DataFrame with some normally distributed data

In [13]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.030276,-0.010513,0.074846,0.002399
std,1.000459,1.013562,0.997122,0.961271
min,-3.590644,-2.921255,-3.169070,-2.743724
25%,-0.618464,-0.650942,-0.598249,-0.631063
50%,0.047743,-0.009134,0.107146,0.023935
75%,0.648149,0.613163,0.779508,0.644841
max,3.401316,3.448640,2.963801,3.064454


In [5]:
# find values in one of the columns exceeding 3 in absolute value
col = data[1]
mask = np.abs(col) > 3
col[mask]

221   -3.022271
405    4.198726
504    3.079752
878   -3.143327
Name: 1, dtype: float64

In [16]:
# select all rows having a value exceeding 3 or –3
mask = (np.abs(data) > 3).any(axis=1)
data[mask]

,0,1,2,3
44,-3.590644,0.483723,0.597334,-0.113335
146,3.401316,0.620458,0.694441,0.885028
191,0.182526,3.014460,-0.460120,0.669391
289,3.286390,-0.072454,-1.224102,-0.133124
391,-3.375508,0.757386,-1.102932,1.288069
449,-1.150811,0.402884,1.249830,3.064454
490,3.149667,-1.843014,0.646842,1.561300
621,-0.408024,-1.111839,-3.169070,-0.604608
634,-0.491334,3.448640,0.458131,-0.327978
913,-0.447711,-0.993565,-3.129335,-0.689097


In [10]:
# set outliers to 3 or -3 depending on its sign
mask = np.abs(data) > 3
data[mask] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.028671,-0.052557,0.037391,-0.024474
std,1.035967,1.025386,0.986704,0.973797
min,-3.000000,-3.000000,-2.914195,-2.673298
25%,-0.673349,-0.743869,-0.632024,-0.644147
50%,0.035445,-0.043571,0.037706,-0.037294
75%,0.719953,0.673910,0.697173,0.633008
max,3.000000,3.000000,3.000000,2.835202


In [12]:
data[158:]

,0,1,2,3
158,-3.000000,-1.419211,-0.072781,0.578729
159,0.931723,-0.020081,1.151043,2.413031
160,-0.025318,0.114457,1.408342,0.430248
161,0.399470,-0.755520,1.075798,0.450272
162,0.274594,-0.032677,0.740834,0.770259
...,...,...,...,...
995,1.509512,0.250636,-0.767468,0.573295
996,1.427509,0.788751,-0.598128,-0.919592
997,-0.927916,-0.943789,1.629783,0.546883
998,0.259796,-0.759939,0.571305,-0.438748


In [17]:
def replace_outlier(row):
    mask = row > 3
    row[mask] = 1
    mask = row < -3
    row[mask] = -1
    
    return row
    
temp = data.apply(replace_outlier, axis=1)


In [19]:
temp[146:]

,0,1,2,3
146,1.000000,0.620458,0.694441,0.885028
147,-0.588642,-0.975912,-0.214895,-0.631793
148,-1.124098,-0.207502,1.037005,0.683090
149,-0.030182,-1.534372,-0.592177,0.976910
150,-0.168934,-0.470671,0.857389,0.844139
...,...,...,...,...
995,1.146128,-0.799225,-0.552358,0.585808
996,-0.601311,0.231916,2.271889,-0.735813
997,-0.651652,1.289926,0.267799,0.974276
998,-0.614461,1.593172,-0.532977,-1.703560


### Computing Indicator/Dummy Variables
Another type of transformation for statistical modeling or machine learning applica‐
tions is converting a categorical variable into a “dummy” or “indicator” matrix.

column in a DataFrame has k distinct values, you would derive a matrix or Data‐
Frame with k columns containing all 1s and 0s.

pandas has a `get_dummies` function for doing this

In [20]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [22]:
# create dummy variables for column 'key'
pd.get_dummies(df["key"], prefix="key")

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [23]:
# create dummy variables for column 'key'  then add the columns to a variable
dummies = pd.get_dummies(df["key"], prefix="key")

In [25]:
# join the dummies to the Data-Frame
df.join(dummies).drop(columns="key")

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [26]:
# read the dataset 'movies.dat' and display the first 10 rows of it

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv('movies.dat', sep='::', engine="python",
                       header=None, names=mnames)
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Adding dummy variables for each genre requires a little bit of wrangling.

In [33]:
# make a list 'genres' contains all distict genres
all_genres = set()
for g in movies["genres"]:
    all_genres = all_genres.union(g.split("|"))
    
all_genres

{'Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

In [39]:
zero_matrix = np.zeros((movies.shape[0], len(all_genres)), dtype=int)
dummies = pd.DataFrame(zero_matrix, columns=all_genres)
dummies

,Drama,Children's,Western,Horror,Comedy,Crime,Mystery,Adventure,War,Musical,Action,Animation,Thriller,Fantasy,Documentary,Sci-Fi,Film-Noir,Romance
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3879,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3880,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3881,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
# create a DataFrame 'dummies' of size (#movies, #genres) and fill it with zeros
movies.shape[0]

3883

In [44]:
# get the genres from the first row of 'movies' DataFrame, then get their indices in the 'dummies' DataFrame
# hint: use 'get_indexer' method

g = movies.loc[0, "genres"]
g.split("|")
dummies.columns.get_indexer(g.split("|"))

array([11,  1,  4], dtype=int64)

In [45]:
# for each value of 'genre' column in 'movies' DataFrame, set the corresponding columns in dummies to 1 

for i, g in enumerate(movies["genres"]):
    columns = dummies.columns.get_indexer(g.split("|"))
    dummies.iloc[i, columns] = 1

dummies

,Drama,Children's,Western,Horror,Comedy,Crime,Mystery,Adventure,War,Musical,Action,Animation,Thriller,Fantasy,Documentary,Sci-Fi,Film-Noir,Romance
0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3879,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3880,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3881,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [48]:
# join the DataFrames 'movies' and 'dummies', then display the first row 
movies.join(dummies.add_prefix("genre_")).drop(columns="genres")

,movie_id,title,genre_Drama,genre_Children's,genre_Western,genre_Horror,genre_Comedy,genre_Crime,genre_Mystery,genre_Adventure,genre_War,genre_Musical,genre_Action,genre_Animation,genre_Thriller,genre_Fantasy,genre_Documentary,genre_Sci-Fi,genre_Film-Noir,genre_Romance
0,1,Toy Story (1995),0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
1,2,Jumanji (1995),0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,4,Waiting to Exhale (1995),1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,Meet the Parents (2000),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3879,3949,Requiem for a Dream (2000),1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3880,3950,Tigerland (2000),1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3881,3951,Two Family House (2000),1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## String Manipulation
Python has long been a popular raw data manipulation language in part due to its
ease of use for string and text processing. Most text operations are made simple with
the string object’s **built-in methods**. 

For more complex pattern matching and text manipulations, **regular expressions** may be needed. 

**pandas** adds to the mix by enabling you to apply string and regular expressions concisely on whole arrays of data,
additionally handling the annoyance of missing data.

### String Object Methods

In [54]:
# convert the string to a list using the ',' as separator
val = 'a,b,  guido'
val.split(",")

['a', 'b', '  guido']

In [55]:
# convert the string to a list using the ',' as separator and remove the extra spaces
pieces = val.split(",")
pieces = [v.strip() for v in pieces]
pieces

['a', 'b', 'guido']

In [56]:
# concatenate the parts back with separator '::'
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [57]:
# concatenate the parts back with separator '::'
"::".join(pieces)

'a::b::guido'

**check**: try the `in` operator and str methods `index`, `find`, `count` and `replace`

![](assets/built-in-str-methods.png)

### Regular Expressions
Regular expressions provide a flexible way to search or match (often more complex)
string patterns in text. A single expression, commonly called a regex, is a string
formed according to the regular expression language.

The `re` module functions fall into three categories: **pattern matching**, **substitution**,
and **splitting**.

In [60]:
import re
text = "foo    bar\t baz  \tqux"
# https://pythex.org



['foo', 'bar', 'baz', 'qux']

suppose we wanted to split a string with a variable number of whitespace characters
(tabs, spaces, and newlines). The regex describing one or more whitespace characters
is \s+:

In [61]:
# split the string depending on the whitespaces
# text.split(" ")
re.split("\s+", text)


['foo', 'bar', 'baz', 'qux']

In [64]:
# find all the string depending on the whitespaces
re.findall("\w+", text)


['foo', 'bar', 'baz', 'qux']

**Note:** Creating a regex object with `re.compile` is highly recommended if you intend to
apply the same expression to many strings; doing so will save CPU cycles

In [65]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [66]:
# get a list of all emails in the text
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

Relatedly, `sub` will return a new string with occurrences of the pattern replaced by the
a new string:


In [68]:
print(regex.sub("SECRET", text))

Dave SECRET
Steve SECRET
Rob SECRET
Ryan SECRET



Suppose you wanted to find email addresses and simultaneously segment each
address into its three components: *username*, *domain name*, and *domain suffix*.

In [70]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [71]:
# find all email in the text
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [75]:
# prefix each segment of the email with a suitable label
print(regex.sub(r"user: \1 @company: \2 .ext: \3", text))

Dave user: dave @company: google .ext: com
Steve user: steve @company: gmail .ext: com
Rob user: rob @company: gmail .ext: com
Ryan user: ryan @company: yahoo .ext: com



![](assets/re-methods.png)

### Vectorized String Functions in pandas

In [46]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data


Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

Series has array-oriented methods for string operations that skip NA values. These are accessed through Series’s **str attribute**

In [47]:
# check if the email is gmail


Regular expressions can be used, too, along with any re options like IGNORECASE

In [49]:
# using the pattern declared earlier, find all parts of each email


In [50]:
# use the match method to check if the field matches an email or not


![](assets/series-str-methods.png)

# Data Aggregation and Group Operations

## GroupBy Mechanics

the term split-apply-combine is used for describing group operations. 

- In the first stage of the process, data contained in a pandas object, whether a Series, Data‐Frame, or otherwise, is **split** into groups based on one or more keys that you provide.
- Once this is done, a function is **applied** to each group, producing a new value. 
- Finally, the results of all those function applications are **combined** into a result object. 

The form of the resulting object will usually depend on what’s being done to the data.

![](assets/group-aggregation.png)

In [51]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randint(0, 10, 5),
                   'data2' : np.random.randint(0, 10, 5)})
df

,key1,key2,data1,data2
0,a,one,1,5
1,a,two,8,9
2,b,one,1,2
3,b,two,0,2
4,a,one,3,6


In [53]:
# group data of column 'data1' by 'key1' then print the groups


In [54]:
# calculate the mean in each group


In [56]:
# group data of column 'data1' by 'key1' and 'key2' then print the groups


In [58]:
# calculate the mean in each group


In [59]:
# unstack the result Series


In [60]:
# try some selection on the result DataFrame


### Iterating Over Groups
The GroupBy object supports iteration, generating a sequence of **2-tuples** containing
the **group name** along with the **chunk of data**.

In [61]:
# group data of DataFrame 'data' by 'key1' then print each group name and data



In [62]:
# group data of DataFrame 'data' by 'key1' and 'key2' then print each group name and data


In [65]:
# group data of DataFrame 'data' by 'key1' then convert it to a dictionary of DataFrames




In [67]:
# group the columns by its datatypes, then print the groups


In [69]:
# loop through the groups and print it


### Selecting a Column or Subset of Columns
Indexing a GroupBy object created from a DataFrame with a column name or array
of column names has the effect of column subsetting for aggregation. This means
that:
```python
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]
```
are syntactic sugar for:
```python
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])
```


In [70]:
# group data of DataFrame 'data' by 'key1' then calculate the mean of column 'data2'



In [73]:
# How to get result as DataFrame GroupBy or Series GroupBy?


### Grouping with Dicts and Series

In [74]:
people = pd.DataFrame(np.random.randint(0, 10, (5,5)),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,8,3.0,0.0,3,6
Steve,3,3.0,2.0,4,7
Wes,9,NaN,NaN,6,4
Jim,5,8.0,4.0,4,7
Travis,8,8.0,2.0,5,2


In [75]:
mapping = {'Joe': 'red', 'Steve': 'red', 'Wes': 'blue',
           'Jim': 'blue', 'Travis': 'red', 'Elon' : 'orange'}

In [77]:
# group and sum the scores of teams red and blue


In [78]:
# convert the dict to a Series, then group and count the scores of teams red and blue



### Grouping with Functions

## independent Practice:
- read the dataset `tips.csv`
- create a new column 'tip_pct', which is tip / total_bill
- replace the short-day name with the full-day name and convert it to upper-case
- calculate the average tip percent for smokers and non-smokers
- calculate the max and average tip percent for each time
- calculate the average tip percent for each day and time
- create dummy variables for the day and time columns


